# Point-by-point refinement notebook  
__Written by Haixing Fang, Jon Wright and James Ball__  
__Date: 21/02/2025__

This notebook will try to perform a point-by-point strain refinement of your pbp index results.  
As with the pbp index, the results of this process are multi-valued.  
You can run 4_visualise to convert the refinement results to an accurate single-valued map.  

### NOTE: It is highly recommended to run this notebook on a Jupyter server with many cores and a lot of RAM.  
The compute_origins() function in particular runs locally and can be compute-intensive for large datasets.  
If this is a big scan (e.g 100 million + 2D peaks), you should definitely refine on the cluster rather than locally.

In [ ]:
import os

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

In [ ]:
exec(open('/data/id11/nanoscope/install_ImageD11_from_git.py').read())

In [ ]:
# this cell is tagged with 'parameters'
# to view the tag, select the cell, then find the settings gear icon (right or left sidebar) and look for Cell Tags

# python environment stuff
PYTHONPATH = setup_ImageD11_from_git( ) # ( os.path.join( os.environ['HOME'],'Code'), 'ImageD11_git' )

# dataset file to import
dset_path = 'si_cube_test/processed/Si_cube/Si_cube_S3DXRD_nt_moves_dty/Si_cube_S3DXRD_nt_moves_dty_dataset.h5'

# which phase to refine
phase_str = 'Si'

# the minimum number of peaks you want a pixel to have to be counted
min_unique = 20

# threshold for whole-sample mask binarisation
manual_threshold = None

# refinement parameters
y0 = 0.0
hkl_tol_origins = 0.05
hkl_tol_refine = 0.1
hkl_tol_refine_merged = 0.05
ds_tol = 0.006
ifrac = 6e-3
rings_to_refine = None  # can be a list of rings
set_mask_from_input = False  # do we mask just from the min_unique value?
use_cluster = False

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from ImageD11.grain import grain
from ImageD11 import unitcell
import ImageD11.sinograms.dataset
from ImageD11.sinograms.point_by_point import PBPMap, PBPRefine
from ImageD11.sinograms.tensor_map import TensorMap
from ImageD11.nbGui import nb_utils as utils

%matplotlib ipympl

# Load data
## Dataset

In [ ]:
ds = ImageD11.sinograms.dataset.load(dset_path)
print(ds)

## Phases
If the parameter file was a json, we can access the unit cells via `ds.phases.unitcells`

In [ ]:
# load phases from parameter file

ds.phases = ds.get_phases_from_disk()
ds.phases.unitcells

In [ ]:
ref_ucell = ds.phases.unitcells[phase_str]
print(ref_ucell)

## Point-by-point map

In [ ]:
pmap = PBPMap(ds.pbpfile.replace('.txt', f'_{phase_str}.txt'))
pmap.plot_nuniq_hist()

In [ ]:
# choose the minimum number of peaks you want a pixel to have to be counted
pmap.choose_best(min_unique)
pmap.plot_best(min_unique)

## Peaks

In [ ]:
cf_2d = ds.get_cf_2d()
ds.update_colfile_pars(cf_2d)
print(f"Read {cf_2d.nrows} 2D peaks")

# Refinement

In [ ]:
# set up a refinement manager object
refine = PBPRefine(dset=ds, y0=y0, hkl_tol_origins=hkl_tol_origins, hkl_tol_refine=hkl_tol_refine, hkl_tol_refine_merged=hkl_tol_refine_merged, ds_tol=ds_tol, ifrac=ifrac, phase_name=phase_str, forref=rings_to_refine)
# change the default paths of the refinement manager to append the phase name
refine.own_filename = os.path.splitext(refine.own_filename)[0] + f'_{phase_str}.h5'
refine.icolf_filename = os.path.splitext(refine.icolf_filename)[0] + f'_{phase_str}.h5'
refine.pbpmap_filename = os.path.splitext(refine.pbpmap_filename)[0] + f'_{phase_str}.h5'
refine.refinedmap_filename = os.path.splitext(refine.refinedmap_filename)[0] + f'_{phase_str}.h5'

In [ ]:
# tell it which point-by-point map we are refining
refine.setmap(pmap)

# or load from disk:
# refine.loadmap()

In [ ]:
# choose 2D peaks to refine with
refine.setpeaks(cf_2d)

# or load from disk:
# refine.loadpeaks()

In [ ]:
# plot the peaks you selected
refine.iplot()

In [ ]:
# generate a single-valued map to refine on
refine.setsingle(refine.pbpmap, minpeaks=min_unique)

In [ ]:
# set whole-sample mask to choose where to refine
# if use_singlemap is true, we will generate a mask simply based on where self.singlemap > min_unique
refine.setmask(manual_threshold=manual_threshold, doplot=True, use_singlemap=set_mask_from_input)

# Compute peak diffraction origins

In [ ]:
# compute diffraction origins - these will be added as a column to refine.icolf
# will then save the new column to disk to avoid re-computation
refine.get_origins()

# Run refinement

In [ ]:
# run the refinement
# if compute_origins took more than a couple of minutes to run, I suggest setting use_cluster=True below
# otherwise if you asked for lots of cores and RAM on this Jupyter instance, you can run it locally (use_cluster=False)
refine.run_refine(use_cluster=use_cluster, pythonpath=PYTHONPATH)

# Export

In [ ]:
# make sure refinement is saved to disk

refine.to_h5()

ds.save()